# Calibrating Material Strength Models with IMPALA
This document summarizes the current IMPALA functionality for calibrating material strength models using the model class **ModelMaterialStrength** for Hopkinson Bar and Quasistatic experiments. This class assumes the observed data for a given experiment consist of an initial temperature (Kelvin), a strain rate (per microseconds), and vectors with strains (unitless) and stress values (megabars).

**Note:** We provide suggested uits for each of the inputs, but the code is written such that users can enter data and corresponding model constants in whatever units they want, provided that the units are all internally consistent. 


### About ModelMaterialStrength

Models of class **ModelMaterialStrength** are defined using the following physics inputs:

1. **temps**    :   a list of temperatures indexed by experiment (Kelvin)
2. **edots**    :   list of strain rates indexed by experiment (per microsecond)
3. **consts**   :   a dictionary of constants for the material stress model. See next section for more details. 
4. **strain_histories**     :   a list of strain histories for Hopkinson Bar/Quasistatic experiments (unitless between 0 and 1, relative to starting length)
5. **flow_stress_model**    :   model for flow/yield stress. Options include constant (perfect plasticity), Johnson-Cook, Preston-Tonks-Wallance, and Stein
6. **melt_model**           :   model for melt temperature as a function of density. 
7. **shear_model**          :   model for shear modulus as a function of density and temperature
8. **specific_heat_model**  :   model for specific heat as a function of temperature
9. **density_model**        :   model for density as a function of temperature

When the calibration model is set up, stress observations are in terms of megabars. 

### Listing Options for Submodels

Many different flow stress, melt, shear, specific heat, and density models have been implemented, and users can specify which of models that make sense for their material of interest. Each specified model is associated with additional constants, which are specified in the consts dictionary in the **ModelMaterialStrength** call. Alternatively, users can specify their own functions for these different models. These functions should follow the input/output structure of the options already implemented in IMPALA, and corresponding constants should be added on the constants dictionary in **ModelMaterialStrength**. 

Below, we list the options currently implemented for each model. The model structure and corresponding constants can be accessed using **showdef_ModelMaterialStrength(str_model_name)**.

In [6]:
import impala.superCal.models_withLik as ml

print(ml.getoptions_ModelMaterialStrength()["flow_stress_model"])
print(ml.getoptions_ModelMaterialStrength()["melt_model"])
print(ml.getoptions_ModelMaterialStrength()["shear_model"])
print(ml.getoptions_ModelMaterialStrength()["specific_heat_model"])
print(ml.getoptions_ModelMaterialStrength()["density_model"])

['Constant_Yield_Stress', 'JC_Yield_Stress', 'PTW_Yield_Stress', 'Stein_Flow_Stress']
['BGP_Melt_Temperature', 'Constant_Melt_Temperature', 'Linear_Melt_Temperature', 'Quadratic_Melt_Temperature']
['BGP_PW_Shear_Modulus', 'Constant_Shear_Modulus', 'Linear_Cold_PW_Shear_Modulus', 'Quadratic_Cold_PW_Shear_Modulus', 'Simple_Shear_Modulus', 'Stein_Shear_Modulus']
['Constant_Specific_Heat', 'Linear_Specific_Heat', 'Piecewise_Linear_Specific_Heat', 'Piecewise_Quadratic_Specific_Heat', 'Quadratic_Specific_Heat']
['Constant_Density', 'Cubic_Density', 'Linear_Density', 'Quadratic_Density']


In [10]:
print(ml.showdef_ModelMaterialStrength("Piecewise_Quadratic_Specific_Heat"))

class Piecewise_Quadratic_Specific_Heat(BaseModel):
    """
   Piecewise Quadratic Specific Heat Model
   Cv (T) = c0_0 + c1_0 * T + c2_0 * T**2 for T<=T_t
   Cv (T) = c0_1 + c1_1 * T + c2_1 * T**2 for T>T_t          
   """
    consts = ['T_t','c0_0', 'c1_0', 'c2_0', 'c0_1', 'c1_1', 'c2_1']
    def value(self, *args):
        tnow=self.parent.state.T
        pow_0_coeff = np.repeat(self.parent.parameters.c0_0,len(tnow))
        pow_1_coeff = np.repeat(self.parent.parameters.c1_0,len(tnow))
        pow_2_coeff = np.repeat(self.parent.parameters.c2_0,len(tnow))
        
        pow_0_coeff[np.where(tnow > self.parent.parameters.T_t)] =  self.parent.parameters.c0_1
        pow_1_coeff[np.where(tnow > self.parent.parameters.T_t)] =  self.parent.parameters.c1_1
        pow_2_coeff[np.where(tnow > self.parent.parameters.T_t)] =  self.parent.parameters.c2_1
        
        cnow = pow_0_coeff + pow_1_coeff * tnow + pow_2_coeff * tnow * tnow
        return cnow

None
